In [1]:
#KNN source code
class KNN:
    n_neighbors=None
    weights=None
    metric=None
    def __init__(self,n_neighbors,weights='uniform',metric_p=2):
        self.n_neighbors= n_neighbors
        self.weights=weights
        self.metric_p=metric_p
#KNN.fit
    def fit(self,x_train,y_train):
        self.x_train=x_train
        self.y_train=y_train
#KNN.predict
    def predict(self,x_test,y_test):
        import pandas as pd 
        import numpy as np
        n_features=len(self.x_train[0])
        train_size=len(self.x_train)
        test_size=len(x_test)
        catagories=np.unique(self.y_train)
        n_catagories=len(catagories)
#Build an empty matrix to contain all the calculated data
        data=np.full((test_size,n_features+2+train_size),np.nan)
#Fill the empty matrix with x_test and y_test
        data[:,:n_features]=x_test
        data[:,n_features]=y_test
# Calculate the distance between test_points and train_points and fill the empty matrix with them
        for i in range(test_size):
            for j in range(train_size):
                data[i,j+n_features+2]=(np.sum((np.abs(self.x_train[j]-x_test[i]))**self.metric_p))**(1/self.metric_p)
# Sort the distance vectors and find out the disatnces and catagories of the closet n_neighbors train points
            distance_catagory_top=np.concatenate((data[i,n_features+2:][:,np.newaxis],self.y_train[:,np.newaxis]),axis=1)
            distance_catagory_top=distance_catagory_top[distance_catagory_top[:,0].argsort()][:self.n_neighbors]
# Find out the predictions of test points through distance_catagory_top
            unique,counts=np.unique(distance_catagory_top[:,-1],return_counts=True)
            if self.weights=='uniform':
                data[i,n_features+1]=unique[counts==np.max(counts)]
            elif self.weights=='distance':
                distance_weight=1/distance_catagory_top[:,0]
                sum_distance_weight=np.zeros(len(unique))
                for k in range(len(unique)):
                    sum_distance_weight[k]=np.sum(distance_weight[distance_catagory_top[:,-1]==unqiue[k]])
                data[i,n_features+1]=unique[sum_distance_weight==np.max(sum_distance_weight)]
            else:
                print("Error: input of 'weights' argument is wrong, it should either 'uniform' or 'distance'.")      
#Define attribuions and output the prediction results
        self.data=data
        self.y_pred=data[:,[n_features,n_features+1]]
        self.pred=data[:,n_features+1]
        print(data[:,n_features+1])     
#KNN.score
    def score(self,x_test,y_test):
        import pandas as pd 
        import numpy as np
        n_features=len(self.x_train[0])
        judge=y_test==self.pred
        unique,counts=np.unique(judge,return_counts=True)
#Avoid the situation that counts[unique==False] or counts[unique==True] becomes empty when true or false not in judge causing empty score.
        if True not in judge:
            score=0.0
        elif False not in judge:
            score=1.0
        else:
            score=counts[unique==True]/(counts[unique==False]+counts[unique==True])
        return print(score)
#KNN.classfication_report
#Calulate the confusion matrics
    def classification_report(self,x_test,y_test):
        import pandas as pd 
        import numpy as np
        unique,counts=np.unique(y_test,return_counts=True)
        n_features=len(self.x_train[0])
        n_catagories=len(unique)
        confusion_matrix=np.full((n_catagories,2,2),np.nan)
        tp=np.full(n_catagories,np.nan)
        fp=np.full(n_catagories,np.nan)
        fn=np.full(n_catagories,np.nan)
        tn=np.full(n_catagories,np.nan)
        for i in range(n_catagories):
            tp[i]=len(self.y_pred[(y_test==unique[i]) & (self.pred==unique[i])])/len(y_test[y_test==unique[i]])
            fp[i]=len(self.y_pred[(y_test!=unique[i]) & (self.pred==unique[i])])/len(y_test[y_test!=unique[i]])
            fn[i]=len(self.y_pred[(y_test==unique[i]) & (self.pred!=unique[i])])/len(y_test[y_test==unique[i]])
            tn[i]=len(self.y_pred[(y_test!=unique[i]) & (self.pred!=unique[i])])/len(y_test[y_test!=unique[i]])
            confusion_matrix[i]=np.array([tp[i],fp[i],fn[i],tn[i]]).reshape(2,2)
        precision=tp/(tp+fp)
        recall=tp/(tp+fn)
        F_value=2*precision*recall/(precision+recall)
        judge_b=y_test==self.pred
        unique_d,counts_d=np.unique(judge_b,return_counts=True)
        if True not in judge_b:
            accuracy=0.0
        elif False not in judge_b:
            accuracy=1.0
        else:
            accuracy=counts_d[unique_d==True]/(counts_d[unique_d==False]+counts_d[unique_d==True])
# Make the dataframe of classifcation report
        support=counts
        weight=support/np.sum(counts)
        catagories=unique
        macro_avg=np.array([np.average(precision),np.average(recall),np.average(F_value)]) 
        weighted_avg=np.array([np.sum(weight*precision),np.sum(weight*recall),np.sum(weight*F_value)])
        df_report_a=pd.DataFrame({'catagory':catagories,'precision':precision,'recall':recall,'f1-score':F_value,'support':support})
        df_report_b=pd.DataFrame([['accuracy','','',np.round(accuracy,6)[0],np.sum(counts)],['macro avg',macro_avg[0],macro_avg[1],macro_avg[2],np.sum(counts)]
                    ,['weighted avg',weighted_avg[0],weighted_avg[1],weighted_avg[2],np.sum(counts)]])
        df_report_a.index=[' ',' ',' ']
        df_report_b.columns=[' ',' ',' ',' ',' ']
        df_report_b.index=[' ',' ',' ']
        self.confusion_matrix=confusion_matrix
        return print(df_report_a,df_report_b)

#test
from sklearn.datasets import load_iris
from sklearn.model_selection import train_test_split
x=load_iris().data
y=load_iris().target
x_train,x_test,y_train,y_test=train_test_split(x,y,test_size=0.3,random_state=123)
clf=KNN(5)
clf.fit(x_train,y_train)
clf.predict(x_test,y_test)
clf.score(x_test,y_test)
clf.classification_report(x_test,y_test)

[2. 2. 2. 1. 0. 2. 1. 0. 0. 1. 2. 0. 1. 2. 2. 2. 0. 0. 1. 0. 0. 2. 0. 2.
 0. 0. 0. 2. 2. 0. 2. 2. 0. 0. 1. 1. 2. 0. 0. 1. 1. 0. 2. 2. 2.]
[0.97777778]
  catagory  precision  recall  f1-score  support
         0   1.000000     1.0  1.000000       18
         1   1.000000     0.9  0.947368       10
         2   0.965517     1.0  0.982456       17                                                 
      accuracy                      0.977778  45
     macro avg  0.988506  0.966667  0.976608  45
  weighted avg  0.986973  0.977778  0.981676  45


C:\Users\wyy\AppData\Local\Temp\ipykernel_7192\2384810890.py:38: DeprecationWarning: Conversion of an array with ndim > 0 to a scalar is deprecated, and will error in future. Ensure you extract a single element from your array before performing this operation. (Deprecated NumPy 1.25.)
  data[i,n_features+1]=unique[counts==np.max(counts)]
